In [1]:
import os
from dotenv import load_dotenv
import anthropic
import gradio as gr

/home/filipefreitas/code_helper/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(override=True)

anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

claude = anthropic.Anthropic()

In [3]:
import requests
from bs4 import BeautifulSoup

def get_wazuh_docs(urls: list[str]) -> str:
    return "\n".join(
        line for url in urls for line in BeautifulSoup(requests.get(url).text, "html.parser").get_text().splitlines() if line.strip()
    )


In [4]:

wazuh_docs_urls = ["https://documentation.wazuh.com/current/installation-guide/wazuh-agent/wazuh-agent-package-windows.html", "https://documentation.wazuh.com/current/installation-guide/wazuh-agent/wazuh-agent-package-linux.html", "https://documentation.wazuh.com/current/installation-guide/wazuh-dashboard/step-by-step.html","https://documentation.wazuh.com/current/installation-guide/wazuh-server/step-by-step.html","https://documentation.wazuh.com/current/installation-guide/wazuh-indexer/step-by-step.html","https://documentation.wazuh.com/current/upgrade-guide/upgrading-central-components.html", "https://documentation.wazuh.com/current/user-manual/user-administration/password-management.html"]

SYSTEM_PROMPT = f"You are a helpful IT assistant. You are helping the user how to configure, install and upgrade wazuh. If you don't know an answer, say so. Don't try to make up an answer if you don't know. Make questions to the user do try to help more your accuracy. You must answer and talk in Brazilian Portuguese."
    

In [5]:
from typing import Iterable
from anthropic.types import MessageParam

class AnthropicChatBot:
    std_system_prompt = "You are a helpful assistant. If you don't know an answer, say so. Don't try to make up an answer if you don't know. Make questions to the user do try to help more your accuracy."
    
    def __init__(self, system_prompt: str = std_system_prompt, context: str | None = None) -> None:
        self.context_loaded = False
        self.system_prompt = system_prompt
        self.context = context

    def chat(self, message, history):
        if not self.context_loaded and self.context:
            context_with_label = "\n\nContext: " + self.context + "\n\n"
            message = context_with_label + message
            self.context_loaded = True

        messages: Iterable[MessageParam] = [{"role": msg["role"], "content": msg["content"]} for msg in history]
        messages.append({"role": "user", "content": message})

        result = claude.messages.stream(
            model="claude-sonnet-4-20250514",
            max_tokens=5489,
            temperature=0.7,
            system=self.system_prompt,
            messages=messages,
        )

        with result as stream:
            new_text = ""
            for text in stream.text_stream:
                new_text += text
                yield new_text
    
    def run(self):
        gr.ChatInterface(fn=self.chat, type="messages").launch()

In [ ]:
context = (
                f"There is a context that you must to pay attention, that is the official "
                f"documentation of Wazuh. To answer take note of this context from the documentation, "
                f"ignoring the parts that are not related to Wazuh procedures:\n\n{get_wazuh_docs(wazuh_docs_urls)}"
            )

chatbot = AnthropicChatBot(SYSTEM_PROMPT, context=context)

chatbot.run()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
